In [1]:
import os
import shutil

import fitz
import pandas as pd
import PyPDF2 as p
from tqdm import tqdm
from PyPDF2.errors import PdfReadError

from philaudit import (convert_pages_to_bitmap, get_part3_pgs,
                       isolate_high_image_area_ratio_pdfs)

In [3]:
def find_big_files(image_dir = "./images"):
    all_files = os.listdir(image_dir)
    bmp_files = [file for file in all_files if file.endswith('.bmp')]

    big_files = set()
    for file in tqdm(bmp_files):
        base_name = os.path.splitext(file)[0].split('.pdf_page_')[0]
        img_count = len([img for img in all_files if base_name in os.path.splitext(img)[0].split('.pdf_page_')[0]])

        if img_count > 60:
            big_files.add(base_name)

    return big_files
        
def test_get_part3_pgs(generate_images=False):
        pdf_dir = "./test_pipeline_pdfs/"
        output_dir = "./test_pipeline_images/"
        log_dir = "./test_logs/"
        data = get_part3_pgs(pdf_dir, log_dir)
        if generate_images:
            print("Converting pages to images...", end=" ")
            for _, row in tqdm(data.iterrows(), total=len(data)):
                convert_pages_to_bitmap(row, output_dir)
            print("Done")
            print(f"All done! Now we have {len(os.listdir(output_dir))} images")
        return data
    
def find_pdfs_that_didnt_convert(img_dir = './images/', pdf_dir = './pdf/'):
    imgfiles = set(file.split('_page_')[0] for file in os.listdir(img_dir) if file.endswith('.bmp'))
    pdffiles = set(file for file in os.listdir(pdf_dir) if file.endswith('.pdf'))
    not_converted = list(pdffiles - imgfiles)
    return not_converted

def grab_files_that_didnt_convert(img_dir = './images/', pdf_dir = './pdf/'):
    not_converted = find_pdfs_that_didnt_convert(img_dir, pdf_dir)
    for file in not_converted:
        # move file to test_pipeline_pdfs
        shutil.move(os.path.join(pdf_dir, file), os.path.join('./test_pipeline_pdfs/', file))

In [4]:
for file in find_big_files(image_dir='./test_pipeline_images/'):
    print("Big File: ", file)

0it [00:00, ?it/s]


In [7]:
find_pdfs_that_didnt_convert(img_dir='./test_pipeline_images/', pdf_dir='./pdf/')

['01-Besao2013_Audit_Report.pdf',
 '01_Pilar2011_Audit_Report.pdf',
 '01-Natonin2013_Audit_Report.pdf']